<h1><font color="#113D68" size=6>Deep Learning con Python y Keras</font></h1>

<h1><font color="#113D68" size=5>Parte 3. Multilayer Perceptron</font></h1>

<h1><font color="#113D68" size=4>6. Proyecto de clasificación binaria</font></h1>

<br><br>
<div style="text-align: right">
<font color="#113D68" size=3>Manuel Castillo Cara</font><br>

</div>

---

<a id="indice"></a>
<h2><font color="#004D7F" size=5>Índice</font></h2>

* [0. Contexto](#section0)
* [1. Rendimiento del modelo de red neuronal de referencia](#section1)
* [2. Optimizar el rendimiento con procesamiento de datos](#section2)
* [3. Ajuste de capas y neuronas](#section3)
    * [3.1. Evaluar una topología más pequeña](#section3.1)
    * [3.2. Evaluar una topología más grande](#section3.2)

---
<a id="section0"></a>
# <font color="#004D7F" size=6> 0. Contexto</font>

En este tutorial del proyecto, descubrirá cómo utilizar eficazmente la biblioteca de Keras en su proyecto de aprendizaje automático trabajando paso a paso en un proyecto de clasificación binaria. Después de completar este tutorial paso a paso, sabrá:
* Cómo cargar datos de entrenamiento y ponerlos a disposición de Keras.
* Cómo diseñar y entrenar una red neuronal para datos tabulares.
* Cómo evaluar el rendimiento de un modelo de red neuronal en Keras sobre datos invisibles.
* Cómo realizar la preparación de datos para mejorar la habilidad al usar redes neuronales.
* Cómo sintonizar la topología y configuración de redes neuronales en Keras.

In [1]:
import tensorflow as tf
# Eliminar warning
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section1"></a>
# <font color="#004D7F" size=6>1. Rendimiento del modelo de red neuronal de referencia</font>

En este problema vamos a utilizar un problema de clasificación binaria como es Sonar en el cual los resultados de Accuracy rondan el 84%.

Creemos un modelo de referencia y un resultado para este problema. Comenzaremos importando todas las clases y funciones que necesitaremos.

Ahora podemos cargar el conjunto de datos usando Pandas y dividir las columnas en 60 variables de entrada _(X)_ y 1 variable de salida _(Y)._ Usamos Pandas para cargar los datos porque maneja fácilmente cadenas (la variable de salida), mientras que intentar cargar los datos directamente usando NumPy sería más difícil.

<div class="alert alert-block alert-info">
    
<i class="fa fa-info-circle" aria-hidden="true"></i>
Más información sobre el dataset [Sonar](https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks))

In [1]:
# Binary Classification with Sonar Dataset: Baseline
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
# load dataset
dataframe = pd. read_csv("Datasets/sonar.csv", header=None)
dataset = dataframe.values
# split into input (X) and output (Y) variables
X = dataset[:,0:60].astype(float)
Y = dataset[:,60]

/Users/manwest/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


La variable de salida son de tipo string. Debemos convertirlos en valores enteros 0 y 1. Podemos hacer esto usando la clase `LabelEncoder` de scikit-learn. Esta clase modelará la codificación requerida usando todo el conjunto de datos a través de la función `fit()`, luego aplicará la codificación para crear una nueva variable de salida usando la función `transform()`.

In [4]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
encoded_Y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

Ahora estamos listos para crear nuestro modelo de red neuronal usando Keras. Vamos a utilizar scikit-learn para evaluar el modelo mediante la validación cruzada. 

Para usar modelos de Keras con scikit-learn, debemos usar el contenedor `KerasClassifier`. Esta clase toma una función que crea y devuelve nuestro modelo de red neuronal. También toma argumentos que pasará a la llamada a `fit()` como el número de épocas y el tamaño del batch. 

Comencemos por definir la función que crea nuestro modelo de línea de base. 
1. Nuestro modelo tendrá una única capa oculta completamente conectada con el mismo número de neuronas que las variables de entrada. Este es un buen punto de partida predeterminado al crear redes neuronales en un problema nuevo.
2. Se utilizara la función de activación ReLu. 
3. La capa de salida contiene una sola neurona para hacer predicciones. Utiliza la función de activación Sigmoidal para producir una salida de probabilidad en el rango de 0 a 1 que puede convertirse fácil y automáticamente en valores de clase nítidos. 
4. Se usará la función de pérdida logarítmica (`binary_crossentropy`) durante el entrenamiento, la función de pérdida preferida para problemas de clasificación binaria. 
6. El modelo también utiliza el eficiente algoritmo de optimización Gradiente Descendiente de Adam y las métricas de precisión se recopilarán cuando se entrene el modelo.

In [5]:
# baseline model
def create_baseline():
    # create model
    model = Sequential()
    model.add(Dense(60, input_dim=60, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

Ahora es el momento de evaluar este modelo. Pasamos el número de épocas de entrenamiento al `KerasClassifier`, nuevamente usando valores predeterminados. 

La salida detallada también se desactiva dado que el modelo se creará 10 veces para realizar la validación cruzada de 10 veces.

In [7]:
# evaluate model with standardized dataset
estimator = KerasClassifier(model=create_baseline, epochs=50, batch_size=5, verbose=0)
kfold = StratifiedKFold(n_splits=5, shuffle=True)
results = cross_val_score(estimator, X, encoded_Y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/Users/manwest/Library/Python/3.9/lib/python/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/

Baseline: 79.87% (6.38%)


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section2"></a>
# <font color="#004D7F" size=6>2. Optimizar el rendimiento con procesamiento de datos</font>

Los modelos de redes neuronales son especialmente adecuados para tener valores de entrada consistentes, tanto en escala como en distribución. Un esquema de preparación de datos efectivo para datos tabulares al construir modelos de redes neuronales es la **estandarización**, i.e., el valor medio de cada atributo sea 0 y la desviación estándar sea 1. Esto preserva las distribuciones gaussianas y similares a las de Gauss mientras normaliza las tendencias centrales para cada atributo.

Podemos usar scikit-learn con la clase `StandardScaler`. En lugar de realizar la estandarización en todo el conjunto de datos, es una buena práctica entrenar el procedimiento de estandarización en los datos de entrenamiento dentro del pase de una ejecución de validación cruzada y usar la instancia de estandarización entrenada para preparar el fold de test no etiquetada. Esto hace que la estandarización sea un paso en la preparación del modelo en el proceso de validación cruzada y evita que el algoritmo tenga conocimiento de datos no etiquetados durante la evaluación, conocimiento que podría pasar del esquema de preparación de datos como una distribución más nítida.

Podemos lograr esto en scikit-learn usando una clase `Pipeline`. 

La ejecución de este ejemplo proporciona los resultados a continuación. Vemos un aumento pequeño pero muy agradable en la precisión media.

In [9]:
# Binary Classification with Sonar Dataset: Standardized
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
# evaluate baseline model with standardized dataset
estimators = []
estimators.append(('standardize', StandardScaler()))
estimators.append(('mlp', KerasClassifier(build_fn=create_baseline, epochs=50, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

/Users/manwest/Library/Python/3.9/lib/python/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/

/Users/manwest/Library/Python/3.9/lib/python/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/

Standardized: 84.17% (5.98%)


---
<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<a id="section3"></a>
# <font color="#004D7F" size=6>3. Ajuste de capas y neuronas</font>

Hay muchas cosas para optimizar en una red neuronal, como la inicialización de los pesos, las funciones de activación, el procedimiento de optimización, etc. Un aspecto que puede tener un efecto descomunal es la estructura de la red neuronal en sí, llamada topología de red. En esta sección echamos un vistazo a dos experimentos sobre la estructura de la red: hacerla más pequeña y hacerla más grande. Estos son buenos experimentos para realizar al optimizar una red neuronal en su problema.

<a id="section3.1"></a>
# <font color="#004D7F" size=5>3.1. Evaluar una topología más pequeña</font>

Sospecho que hay mucha redundancia en las variables de entrada para este problema. Los datos describen la misma señal desde diferentes ángulos. Quizás algunos de esos ángulos sean más relevantes que otros. Podemos forzar un tipo de extracción de características por parte de la red restringiendo el espacio de representación en la primera capa oculta.

En este experimento, tomamos nuestro modelo de línea base con 60 neuronas en la capa oculta y lo reducimos a la mitad a 30. Esto ejercerá presión sobre la red durante el entrenamiento para seleccionar la estructura más importante en los datos de entrada para modelar. También estandarizaremos los datos como en el experimento anterior con la preparación de datos e intentaremos aprovechar el pequeño aumento en el rendimiento.

/Users/manwest/Library/Python/3.9/lib/python/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/

Smaller: 76.49% (6.13%)


Podemos ver que tenemos un aumento muy leve en el Accuracy medio y una reducción importante en la desviación estándar (dispersión promedio). Este es un gran resultado porque lo estamos haciendo un poco mejor con una topología de la mitad del tamaño, que a su vez toma la mitad de tiempo para entrenar.

<a id="section3.2"></a>
# <font color="#004D7F" size=5>3.2. Evaluar una topología más grande</font>

Una topología de red neuronal con más capas ofrece más oportunidades para que la red extraiga características clave y las recombine de formas útiles no lineales. Aquí, agregamos una nueva capa (una línea) a la red que introduce otra capa oculta con 30 neuronas después de la primera capa oculta. Nuestra red ahora tiene la topología:
```
    60 inputs -> [60 -> 30] -> 1 output
``` 


La idea aquí es que la red tiene la oportunidad de modelar todas las variables de entrada antes de ser atascada y obligada a reducir a la mitad la capacidad de representación, como hicimos en el experimento anterior con la red más pequeña. En lugar de exprimir la representación de las entradas en sí, tenemos una capa oculta adicional para ayudar en el proceso.

/Users/manwest/Library/Python/3.9/lib/python/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/Users/manwest/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/

Larger: 84.64% (6.27%)


Podemos ver que no obtenemos un grandes cambios en el rendimiento del modelo. Esto puede ser un ruido estadístico o una señal de que se necesita más entrenamiento.

Con un mayor ajuste de aspectos como el algoritmo de optimización y el número de épocas de entrenamiento, se espera que sean posibles más mejoras. ¿Cuál es la mejor puntuación que puede lograr en este conjunto de datos?

<div style="text-align: right"> <font size=5> <a href="#indice"><i class="fa fa-arrow-circle-up" aria-hidden="true" style="color:#004D7F"></i></a></font></div>

---

<div style="text-align: right"> <font size=6><i class="fa fa-coffee" aria-hidden="true" style="color:#004D7F"></i> </font></div>